In [1]:
import requests_html
import pandas as pd
import re

from bs4 import BeautifulSoup

In [2]:
holiday_list = []

r = requests_html.HTMLSession()
headers = {'Accept-Language': 'en-US,en;q=0.5'}

for year in range(2017, 2022):
    print(f'Getting data for {year}')
    html = r.get(f'https://www.timeanddate.com/holidays/poland/{year}', headers=headers)
    
    if html.status_code==200:
        
        soup = BeautifulSoup(html.text, "lxml")
        
        tbl_rows = soup.find(name="table", attrs={"id" : "holidays-table"}).\
                    find_all(name="tr", attrs={'id' : re.compile(r'^tr\d')})
        
        for row in tbl_rows:
            holiday_list.append({
                'date' : row.find('th').text + f' {year}',
                'holiday_name' : row.find('a').text,
                'holiday_type' : row.find_all('td')[-1].text
            })

Getting data for 2017
Getting data for 2018
Getting data for 2019
Getting data for 2020
Getting data for 2021


In [3]:
holiday_df = pd.DataFrame(holiday_list)
holiday_df['date'] = pd.to_datetime(holiday_df['date'])

In [4]:
holiday_df

,date,holiday_name,holiday_type
0,2017-01-01,New Year's Day,National holiday
1,2017-01-06,Epiphany,National holiday
2,2017-02-14,Valentine's Day,Observance
3,2017-03-20,March Equinox,Season
4,2017-04-14,Good Friday,Observance
...,...,...,...
127,2021-12-24,Christmas Eve,Observance
128,2021-12-25,Christmas Day,National holiday
129,2021-12-26,Second Day of Christmas,National holiday
130,2021-12-27,National Day of the Victorious Greater Poland ...,Observance


In [5]:
holiday_df.to_csv('poland_holidays.csv', index=False)